## Notebook used with redback to generate ToO style plots for the publication version of the ToO report

### You will need to install Redback. Instructions available at https://redback.readthedocs.io/en/latest/. I suggest installing from source via GitHub.

Notebook Template Author: Meet Vyas <mjvyas2002@gmail.com>

Code reviewed by Igor Andreoni and Ritwik Sharma 

##  Instructions to create the plots using Google Colab

> ### 1. Clone the repository in your colab environment

In [ ]:
!git clone https://github.com/nikhil-sarin/redback

> ### 2. Make the cloned repository your current directory

In [ ]:
cd redback/

> ### 3. Install the requirements for redback 

In [ ]:
!pip install -r requirements.txt

> ### 4. Also install the additional requirements 

In [ ]:
!pip install lalsuite nestle

> ### 5. Redback uses latex for character typesetting and therefore download latex for google colab

In [ ]:
!sudo apt-get install -y texlive texlive-latex-extra texlive-fonts-recommended dvipng cm-super

### This ends the google colab instructions, follow along on your local machine from here

> ### Import all the required libraries and dependencies

In [ ]:
import redback
import pandas as pd
from redback.simulate_transients import SimulateOpticalTransient
import matplotlib.pyplot as plt
import numpy as np

### We use the Golden Strategy which comprises of a 3 filter deep strategy. This takes the form of a dataframe which specifies time from transient onset, right ascension, declination, filters and five sigma depth mags. The three cases for the Golden Strategy are given as follows 

> Night 0 timing considerations: we calculate that it will take 95 minutes (1.59 hrs) to scan a 100 sq deg skymap and the scheduling of the three epochs on Night 0 will depend on the observability window of the sky map (Twin ). Here we provide a timing strategy:
> > ➢ Case 1: If Twin >=3*1.59 = 4.76 hrs; do all 3 scans, with the gap between them of ((Twin - 3*1.59)/2)
> > 
> > ➢ Case 2: Else if 3.2 < Twin < 4.7 hrs; do 2 scans. The start time of the visits is separated by (Twin - 2*1.59), which is sufficient for asteroid rejection and short timescale evolution.
> > 
> > ➢ Case 3: Else if Twin < 3.2 hrs then do 1x1.59 hrs scan + followed by 1 scan (0.53 hrs) in r-band only after the full filter complement is complete, to ensure asteroid rejection is reliable.

For the pointings file we have used the 5 sigma magnitudes calculated from etc.py based on the criteria for 120 seconds of exposure time on day 0 and 180 seconds on Day 1,2, and 3 as given in the Report, attached below for reference 

 > ➢ Night 0: three scans across the skymap in 3 filters with 120-sec exposure for each filter
gri8
Each scan is estimated to take ~95 minutes.
 > 
 > ➢ Night 1,2,3: one scan across the skymap in two filters (likely r+i) with 180 seconds per
visit. Each night is estimated to take ~92 minutes

 ### The manual sky pointings csv files need to be read using the pandas library 

In [ ]:
# Load the csv file for case 1 of the Golden Strategy
pointings_case_1 = pd.read_csv("Golden_Strategy_Case_1.csv")
print(pointings_case_1)

# Load the csv file for case 2 of the Golden Strategy
pointings_case_2 = pd.read_csv("Golden_Strategy_Case_2.csv")
print(pointings_case_2)

# Load the csv file for case 3 of the Golden Strategy
pointings_case_3 = pd.read_csv("Golden_Strategy_Case_3.csv")
print(pointings_case_3)

 ### Below we will load the parameters for the kilonova model. The model takes redshift as a parameter therefore we will convert the luminosity distance to redshift which we do using the planck18 function using astropy 

In [1]:
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u
from astropy.cosmology import z_at_value

# Ask the user for the distance in Mpc
distance_mpc = float(input("Enter the distance in Mpc: ")) * u.Mpc

# Calculate the redshift corresponding to the user-provided distance
redshift = z_at_value(cosmo.luminosity_distance, distance_mpc)
print(f"The redshift corresponding to a distance of {distance_mpc} is: {redshift:.5f}")


The redshift corresponding to a distance of 350.0 Mpc is: 0.07482 redshift


 ### We are using the One Component kilonova model for the lightcurve simulations for a Faint kilonova from the redback library .


> First we update the parameters for the model at a luminosity distance of 350 Mpc

In [ ]:
# Set model-specific keyword arguments.
model_kwargs = {'output_format': 'magnitude'}

# Specify a model. Here, any redback model can be referred to as a string. 
# If the user has their own model, they can pass a function instead. 
# There are over 100 models in redback, including options for kilonovae, GRB afterglows,
# supernovae, TDEs, and other transient events.
model = 'one_component_kilonova_model'

# Load and sample from the default prior for the chosen model, generating parameters for a random kilonova.
# This creates a base set of parameters that can be adjusted as needed.
parameters = redback.priors.get_priors(model=model).sample()

# We can also set the parameters manually. For the arxiv plots we use these parameters for the 350 Mpc kilonova event.
parameters['mej'] = 0.05                 # Ejecta mass in solar masses
parameters['t0_mjd_transient'] = 0       # Start time of the transient in MJD (Modified Julian Date)
parameters['redshift'] = 0.074816848     # Redshift for the kilonova event at 350 Mpc 
parameters['t0'] = parameters['t0_mjd_transient']  # Set the transient's starting time
parameters['temperature_floor'] = 3000   # Minimum temperature in Kelvin (floor value) to avoid extremely low temperatures in the model
parameters['kappa'] = 1                  # Gray opacity, representing the opacity of the ejecta material
parameters['vej'] = 0.2                  # Minimum initial velocity of the ejecta in units of c (speed of light)
parameters['ra'] = 1.0                   # Right ascension in degrees for the kilonova's location
parameters['dec'] = 1.5                  # Declination in degrees for the kilonova's location
print(parameters)


### One can check the validity of the parameters used based on the priors defined in the library, the priors for the one component kilonova can be loaded as

In [ ]:
priors = redback.priors.get_priors(model=model)
priors

> Parameters for the model at a luminosity distance of 700 Mpc

In [ ]:
model_kwargs = {'output_format': 'magnitude'}
# Any redback model can be referred to as a string. 
# If the user has their own model, they can pass a function here instead. 
# There are over a 100 models implemented in redback, lots of models for kilonovae, GRB afterglows, 
# supernovae, TDEs and other things
model = 'one_component_kilonova_model'
# Load the default prior for this model in redback and sample from it to get 1 set of parameters. 
# We can sample from the default prior for this model for a random kilonova. 
parameters = redback.priors.get_priors(model=model).sample()

# We fix a few parameters here to create a nice looking kilonova. 
# You can change any of the parameters here or add additional keyword arguments 
# to change some physical assumptions. Please refer to the documentation for this and units etc
parameters['mej'] = 0.05 # Ejecta mass in solar masses
parameters['t0_mjd_transient'] = 0 # Start time of the transient in MJD (Modified Julian Date)
parameters['redshift'] = 0.14309633 # Redshift for 700 Mpc
parameters['t0'] = parameters['t0_mjd_transient'] # Set the transient's starting time
parameters['temperature_floor'] = 3000 # Minimum temperature in Kelvin (floor value) to avoid extremely low temperatures in the model
parameters['kappa'] = 1 # Gray opacity, representing the opacity of the ejecta material
parameters['vej'] = 0.2 # Minimum initial velocity of the ejecta in units of c (speed of light)
parameters['ra'] = 1.0 # Right ascension in degrees for the kilonova's location
parameters['dec'] = 1.5 # Declination in degrees for the kilonova's location
print(parameters)

> ### Simulating Kilonovae for Different Cases

We now simulate kilonovae for the 3 cases discussed above using the specified parameters for either the 350 Mpc or the 700 Mpc case, depending on which code block is used. This simulation utilizes the strategy designed in the corresponding CSV files.


In [ ]:
# Simulate the transient for the Faint Kilonova case 1
kn_sim_case_1 = SimulateOpticalTransient.simulate_transient(model='metzger_kilonova_model',
                                       parameters=parameters, pointings_database=pointings_case_1,
                                       survey=None, model_kwargs=model_kwargs,
                                        end_transient_time=4, snr_threshold=5.0)

# Simulate the transient for the Faint Kilonova case 2
kn_sim_case_2 = SimulateOpticalTransient.simulate_transient(model='metzger_kilonova_model',
                                       parameters=parameters, pointings_database=pointings_case_2,
                                       survey=None, model_kwargs=model_kwargs,
                                        end_transient_time=4, snr_threshold=5.0)

# Simulate the transient for the Faint Kilonova case 3
kn_sim_case_3 = SimulateOpticalTransient.simulate_transient(model='metzger_kilonova_model',
                                       parameters=parameters, pointings_database=pointings_case_3,
                                       survey=None, model_kwargs=model_kwargs,
                                        end_transient_time=4, snr_threshold=5.0)

### We can now print the observations that were simulated to look at the data that was generated. The data contains 

- `time`: 
  - The time of observation in mjd relative to the transient event start.

- `magnitude`: 
  - The apparent AB magnitude of the observed object in the specified filter.

- `e_magnitude`: 
  - The error or uncertainty in the measured magnitude.

- `band`: 
  - The filter used for the observation. (e.g., lsstg, lsstr, lssti, etc.)

- `system`: 
  - The photometric system used for magnitude calculation. In this case, it's the AB magnitude system.

- `flux_density(mjy)`:
  - The observed flux density in millijanskys (mJy).

- `flux_density_error`: 
  - The uncertainty in the flux density measurement (mJy).

- `flux(erg/cm2/s)`: 
  - The observed flux in units of erg/cm²/s.

- `flux_error`: 
  - The uncertainty in the observed flux (erg/cm²/s).

- `time (days)`: 
  - Same as the `time` column but shown with the label "days".

- `detected`: 
  - A binary flag (1 or 0) indicating whether the transient was detected in the observation (1 for detected, 0 for non-detected).

- `limiting_magnitude`: 
  - The limiting magnitude for the observation in the specified band. Observations fainter than this value may not be detected.

### AB Magnitude System

The AB magnitude system is a photometric system used to measure the brightness of astronomical objects. It is defined in terms of the flux density of the source in a specific bandpass. The AB magnitude $m_{\text{AB}}$ is given by the following formula:

$$
m_{\text{AB}} = -2.5 \log_{10} \left( \frac{f_{\nu}}{3631 \ \text{Jy}} \right)
$$


Alternatively, with $f_{\nu}$ still in janskys, the formula can also be written as:

$$
m_{\text{AB}} = -2.5 \log_{10} \left( f_{\nu} \right) + 8.90
$$

Where:
- $m_{\text{AB}}$ is the AB magnitude.
- $f_{\nu}$ is the flux density of the source in a given band, measured in units of $\text{Jy}$ (Jansky, $1 \ \text{Jy} = 10^{-26} \ \text{W} \cdot \text{m}^{-2} \cdot \text{Hz}^{-1}$).
- $3631 \ \text{Jy}$ is the reference flux for an object with zero AB magnitude.

### Explanation of Terms
- **Flux density** ($f_{\nu}$): The amount of energy per unit area per unit time per unit frequency being emitted by an astronomical source, typically in Janskys (Jy) for the AB system.
- **Magnitude**: A logarithmic scale for measuring the brightness of an astronomical object. Lower magnitudes correspond to brighter objects.
- **Reference Flux**: The AB magnitude system is designed so that an object with an equal flux density of $3631 \ \text{Jy}$ in any filter has an AB magnitude of 0.




In [ ]:
# Print the simulated observations for Golden Strategy Case-1
print(kn_sim_case_1.observations)

# Print the simulated observations for Golden Strategy Case-2
print(kn_sim_case_2.observations)

# Print the simulated observations for Golden Strategy Case-3
print(kn_sim_case_3.observations)

### Saving Observations to a File

We can also save the observations to a file using the `save_transient` method. This will save the observations to a CSV file in a `simulated` directory alongside the CSV file specifying the injection parameters.



In [ ]:
# Save the simulated observations for Golden Strategy Case-1
kn_sim_case_1.save_transient(name='Golden_Strategy_Case_1')

# Save the simulated observations for Golden Strategy Case-2
kn_sim_case_2.save_transient(name='Golden_Strategy_Case_2')

# Save the simulated observations for Golden Strategy Case-3
kn_sim_case_3.save_transient(name='Golden_Strategy_Case_3')

> ### Calculating Filter Offsets and Plotting Simulated Light Curves

Now that the data has been simulated and saved, we will define two custom functions. 

The first function will calculate the filter offsets for each scan based on the 5-sigma limiting magnitude of each band. Using the **etc.py** file, it will calculate the resulting exposure time in hours. 

In [ ]:
import matplotlib.ticker as ticker
import etc

def calculate_time_offsets_from_pointings(pointings, airmass=1.0):
    """
    Calculate the filter time offsets from the manual pointings file using the fiveSigmaDepth.
    Handles the filter mapping from 'lsstg', 'lsstr', etc., to 'g', 'r', etc.
    """
    # Define the mapping from 'lsst' filters to 'u', 'g', 'r', 'i', 'z', 'y' as etc.py expects these
    filter_mapping = {
        'lsstu': 'u',
        'lsstg': 'g',
        'lsstr': 'r',
        'lssti': 'i',
        'lsstz': 'z',
        'lssty': 'y'
    }
    
    filter_time_offsets = {}
    
    for index, row in pointings.iterrows():
        # Get the 'lsst' filter
        lsst_filt = row['filter']
        
        # Map the 'lsst' filter to the corresponding 'u', 'g', 'r', etc.
        filt = filter_mapping.get(lsst_filt, lsst_filt)  # Default to the original if no mapping found
        
        depth = row['fiveSigmaDepth']
        
        # Calculate the exposure time using the limiting magnitude (depth)
        exptime = etc.get_exptime(depth, filt, X=airmass)
        
        # Convert exposure time to hours (assuming exptime is in seconds)
        time_offset_hr = exptime / 3600
        
        # Store the offset for each filter, if multiple rows for the same filter, take the latest
        filter_time_offsets[lsst_filt] = time_offset_hr  # Keep lsst format key
    
    return filter_time_offsets

The second function will use these filter offsets, along with a visual hour delay from the transient onset and the start of the observation window, to plot the light curves simulated through the **redback** library in the Rubin ToO format.

In [ ]:
import matplotlib.ticker as ticker

def redback_in_rubin_toO(kn_sim, visual_delay_hr, pointings, filters_color_dict, plot_title, xlim=[0, 4], ylim=[28, 18], n_interp=120):
    """
    Function to plot light curves using simulated kilonova data and filter time offsets.
    
    Parameters:
    - kn_sim: simulated kilonova object from redback
    - visual_delay_hr: visual delay in hours from the transient onset
    - pointings: pointings file with fiveSigmaDepth magnitudes
    - filters_color_dict: dictionary mapping filters to their colors
    - plot_title: title for the plot (should include distance and case information)
    - xlim: x-axis limits
    - ylim: y-axis limits
    - n_interp: number of points for interpolation of the light curve
    """
    
    # Extract observations from kn_sim object for detected and upper limit data
    detected_data = kn_sim.observations[kn_sim.observations['detected'] == 1.0]
    upper_limits = kn_sim.observations[kn_sim.observations['detected'] != 1.0]
    
    # Calculate time offsets from the pointings file for each filter
    filter_time_offsets = calculate_time_offsets_from_pointings(pointings)
    print(filter_time_offsets)

    # Initialize figure
    fig, ax = plt.subplots()
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)

    # Initialize flags for adding detection and upper limit labels only once
    added_detection_label = False
    added_upper_limit_label = False

    # Plot the detected data points (detections) with visual delay and filter time offsets
    for band in filters_color_dict.keys():
        detected_band_data = detected_data[detected_data['band'] == band]
        detected_times = detected_band_data['time (days)']
        detected_magnitudes = detected_band_data['magnitude']
        
        if len(detected_times) > 0:  # Only plot detections if there are any
            # Apply both visual delay and filter-specific time offset
            shifted_times = detected_times + (visual_delay_hr / 24) + (filter_time_offsets.get(band, 0) / 24)
            
            if band == "lsstz":
                if not added_detection_label:
                    plt.scatter(shifted_times, detected_magnitudes, color=filters_color_dict[band],
                                marker='o', s=100, edgecolor='black', linewidth=1.5, label="Det.")
                    added_detection_label = True
                else:
                    plt.scatter(shifted_times, detected_magnitudes, color=filters_color_dict[band],
                                marker='o', s=100, edgecolor='black', linewidth=1.5, label="_nolegend_")
            else:
                plt.scatter(shifted_times, detected_magnitudes, color=filters_color_dict[band],
                            marker='o', s=100, edgecolor='black', linewidth=1.5, label="_nolegend_")

    # Check if the black band has no detections and add a placeholder for the legend
    if "lsstz" not in detected_data['band'].values:
        plt.scatter([], [], color='k', marker='o', s=100, edgecolor='black', linewidth=1.5, label="Det.")

    # Plot the non-detections (upper limits) with visual delay and filter time offsets
    for band in filters_color_dict.keys():
        up = upper_limits[upper_limits['band'] == band]
        up_times = up['time (days)']
        if len(up_times) > 0:
            shifted_times = up_times + (visual_delay_hr / 24) + (filter_time_offsets.get(band, 0) / 24)
            valid_times = shifted_times[shifted_times >= 0]  # Ensure no negative time values
            valid_limits = up['limiting_magnitude'][shifted_times >= 0]

            if band == "lsstz":
                if not added_upper_limit_label:
                    plt.scatter(valid_times, valid_limits, s=100, marker='v', edgecolor='blue', linewidth=1.5,
                                color=filters_color_dict[band], label="UL")
                    added_upper_limit_label = True
                else:
                    plt.scatter(valid_times, valid_limits, s=100, marker='v', edgecolor='blue', linewidth=1.5,
                                color=filters_color_dict[band], label="_nolegend_")
            else:
                plt.scatter(valid_times, valid_limits, s=100, marker='v', edgecolor='blue', linewidth=1.5,
                            color=filters_color_dict[band], label="_nolegend_")

    # Check if the black band has no upper limit and add a placeholder for the legend
    if "lsstz" not in upper_limits['band'].values:
        plt.scatter([], [], color='k', marker='v', s=100, edgecolor='blue', linewidth=1.5, label="UL")

    # Now plot the light curve model
    tt = np.linspace(0, 4, n_interp)  # Adjust the time range for the model
    parameters['output_format'] = 'magnitude' # Ensure the model output is in magnitude

    for band in filters_color_dict.keys():
        parameters['bands'] = band
        out = redback.transient_models.kilonova_models.one_component_kilonova_model(tt, **parameters)
        plt.plot(tt + (visual_delay_hr / 24) + (filter_time_offsets.get(band, 0) / 24), out,
                 color=filters_color_dict[band], alpha=0.8, linewidth=2, label=f'{band[-1]}')

    # Add horizontal dashed lines for each filter's fiveSigmaDepth magnitude as a reference

    plt.axhline(y=24.5, color='r', linestyle='dotted', linewidth=2)
    plt.axhline(y=23, color='yellow', linestyle='dotted', linewidth=2)
    plt.axhline(y=25, color='g', linestyle='dotted', linewidth=2)

    # Get current handles and labels
    handles, labels = ax.get_legend_handles_labels()

    # Filter out unwanted labels (like 'lssti', 'lsstg', 'lsstr'), these were appearing in the final legend when calling the plt.legend function
    filtered_handles = []
    filtered_labels = []
    for handle, label in zip(handles, labels):
        if label not in ['lssti', 'lsstg', 'lsstr']:  # Exclude these labels
            filtered_handles.append(handle)
            filtered_labels.append(label)

    # Rebuild the legend with only the desired items and set the legend text color to black
    plt.legend(filtered_handles, filtered_labels, loc='center left', bbox_to_anchor=(1.05, 0.5), fontsize=10, frameon=True)

    # Set x-axis tick locator to intervals of 1 day
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

    # Customize the plot
    plt.rcParams["font.family"] = "sans-serif"
    plt.rcParams["font.weight"] = "bold"
    plt.xlabel("Time from start of Rubin observing window (days)", fontsize=13)
    plt.ylabel("AB Magnitude", fontsize=13)
    
    # Use the user-provided plot title
    plt.title(plot_title, fontsize=13, weight='bold')

    # Add annotation
    plt.text(3.5 + (visual_delay_hr / 24), 19, "Faint KN", fontsize=20, weight='bold', color='black',
             horizontalalignment='right', verticalalignment='top')

    # Show or save the plot
    plt.show()

### Now we can call the function to plot the lightcurves for different cases of the Golden Strategy 

> Run the below code cell if you are plotting the lightcurves for a luminosity distance of 350 Mpc

In [ ]:
# Plot the light curve for the Faint Kilonova with the Golden Strategy Case 1

plot_title1 = "Gold Events: 3 Filter Deep Strategy (350 Mpc) Case 1"
redback_in_rubin_toO(kn_sim_case_1, visual_delay_hr=6, pointings=pointings_case_1, filters_color_dict={'lsstg':'g', 'lsstu':'b', 'lsstr':'r', 'lsstz':'k', 'lssti':'yellow', 'lssty':'orange'}, plot_title=plot_title1)

# Plot the light curve for the Faint Kilonova with the Golden Strategy Case 2

plot_title2 = "Gold Events: 3 Filter Deep Strategy (350 Mpc) Case 2"
redback_in_rubin_toO(kn_sim_case_2, visual_delay_hr=6, pointings=pointings_case_2, filters_color_dict={'lsstg':'g', 'lsstu':'b', 'lsstr':'r', 'lsstz':'k', 'lssti':'yellow', 'lssty':'orange'}, plot_title=plot_title2)

# Plot the light curve for the Faint Kilonova with the Golden Strategy Case 3

plot_title3 = "Gold Events: 3 Filter Deep Strategy (350 Mpc) Case 3"
redback_in_rubin_toO(kn_sim_case_3, visual_delay_hr=6, pointings=pointings_case_3, filters_color_dict={'lsstg':'g', 'lsstu':'b', 'lsstr':'r', 'lsstz':'k', 'lssti':'yellow', 'lssty':'orange'}, plot_title=plot_title3)


> Run the below code cell if you are plotting the lightcurves for a luminosity distance of 700 Mpc

In [ ]:
# Plot the light curve for the Faint Kilonova with the Golden Strategy Case 1

plot_title4 = "Gold Events: 3 Filter Deep Strategy (700 Mpc) Case 1"
redback_in_rubin_toO(kn_sim_case_1, visual_delay_hr=6, pointings=pointings_case_1, filters_color_dict={'lsstg':'g', 'lsstu':'b', 'lsstr':'r', 'lsstz':'k', 'lssti':'yellow', 'lssty':'orange'}, plot_title=plot_title4)

# Plot the light curve for the Faint Kilonova with the Golden Strategy Case 2

plot_title5 = "Gold Events: 3 Filter Deep Strategy (700 Mpc) Case 2"
redback_in_rubin_toO(kn_sim_case_2, visual_delay_hr=6, pointings=pointings_case_2, filters_color_dict={'lsstg':'g', 'lsstu':'b', 'lsstr':'r', 'lsstz':'k', 'lssti':'yellow', 'lssty':'orange'}, plot_title=plot_title5)

# Plot the light curve for the Faint Kilonova with the Golden Strategy Case 3

plot_title6 = "Gold Events: 3 Filter Deep Strategy (700 Mpc) Case 3"
redback_in_rubin_toO(kn_sim_case_3, visual_delay_hr=6, pointings=pointings_case_3, filters_color_dict={'lsstg':'g', 'lsstu':'b', 'lsstr':'r', 'lsstz':'k', 'lssti':'yellow', 'lssty':'orange'}, plot_title=plot_title6)